In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

In [2]:
df = pd.read_csv('../input/dataset.csv')
df.head()

,WindNumber,Time,WindSpeed,Power,RotorSpeed
0,1,2017/11/1 0:20,4.188333,65.750000,5.779167
1,1,2017/11/1 0:30,4.042500,120.700000,8.636500
2,1,2017/11/1 0:40,4.137368,135.157895,8.668421
3,1,2017/11/1 0:50,4.274737,157.315789,8.727895
4,1,2017/11/1 1:00,4.108947,129.894737,8.656842


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497837 entries, 0 to 497836
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   WindNumber  497837 non-null  int64  
 1   Time        497837 non-null  object 
 2   WindSpeed   497837 non-null  float64
 3   Power       497837 non-null  float64
 4   RotorSpeed  497837 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 19.0+ MB


In [4]:
sub = df[['WindNumber', 'Time']]

In [5]:
df['Power_overload'] = np.where(df['Power'] > 2000, 1, 0)

In [6]:
def RotorSpeed_process(x):
    if x['WindNumber'] in [1, 2, 3, 4, 6, 7, 8, 9, 10] and x['RotorSpeed'] > 16.8:
        return 1
    elif x['WindNumber'] == 5 and x['RotorSpeed'] > 19:
        return 1
    elif x['WindNumber'] == 11 and x['RotorSpeed'] > 14:
        return 1
    elif x['WindNumber'] == 12 and x['RotorSpeed'] > 17:
        return 1
    else:
        return 0

In [7]:
df['RotorSpeed_overload'] = df.apply(RotorSpeed_process, axis=1)

In [8]:
df.drop(['Time'], axis=1, inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497837 entries, 0 to 497836
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   WindNumber           497837 non-null  int64  
 1   WindSpeed            497837 non-null  float64
 2   Power                497837 non-null  float64
 3   RotorSpeed           497837 non-null  float64
 4   Power_overload       497837 non-null  int32  
 5   RotorSpeed_overload  497837 non-null  int64  
dtypes: float64(3), int32(1), int64(2)
memory usage: 20.9 MB


In [10]:
X = np.asarray(df)

In [11]:
clf = IsolationForest(n_estimators=200,
                      max_samples='auto',
                      contamination='auto',
                      behaviour='new',
                      random_state=2020,
                      n_jobs=-1,
                      verbose=0)
clf.fit(X)
y_pred = clf.predict(X)

In [12]:
y_pred = np.where(y_pred == -1, 0, 1)

In [13]:
sub['label'] = y_pred
sub.to_csv('../sub/baseline.csv', index=False)

C:\Users\z\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
